In [1]:
import cv2
import numpy as np

In [2]:
model_name = 'opencv_data/res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'opencv_data/deploy.prototxt.txt'

# 최소 예측 정확도
min_confidence = 0.5
file_name = 'opencv_data/video/obama_01.mp4'

In [3]:
# 분석 모델을 생성한다.
model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)

In [9]:
# 영상을 재생한다.
cap = cv2.VideoCapture(file_name)

while True:
    # 영상 프레임 하나를 읽어온다.
    ret, frame = cap.read()
    
    # 더 이상 읽어온 프레임이 없다면 중단시킨다.
    if frame is None:
        break
        
    # frame의 사이즈를 300x300으로 조정한다.
    a1 = cv2.resize(frame,(300,300))
    # 2진 데이터로 변환한다.
    blob = cv2.dnn.blobFromImage(a1, 1.0, (300,300), (104.0, 177.0, 123.0))
    # 변환된 데이터를 모델에 넣어준다.
    model.setInput(blob)
    
    # 얼굴을 인식한다.
    detections = model.forward()
    #print(detections.shape[2]) # 인식된 얼굴의 수
    
    # 인식된 얼굴의 수만큼 반복한다.
    for i in range(0,detections.shape[2]):
        
        # 얼굴임을 예측한 정확도를 가져온다.
        confidence = detections[0,0,i,2]
        # print(confidence)
        
        # 원본 이미지의 가로, 세로 길이를 구한다.
        height = frame.shape[0]
        width = frame.shape[1]
        
        # 인식된 얼굴의 위치(비율)를 추출한다.
        box1 = detections[0,0,i,3:7]
        # 비율을 실제 조표로 환산한다.
        box2 = box1 * np.array([width, height, width, height])
        x1, y1, x2, y2 = box2.astype('int')
        # print(x1,y1,x2,y2)
        
        # 예측 정확도가 최소 정확도보다 높은 것만 처리한다.
        if confidence > min_confidence:
            # 영역을 표시한다.(사각형을 그린다.)
            cv2.rectangle(frame, (x1,y1),(x2,y2), (0,255,0),2)
            # 예측 정확도를 표시한다.
            text = f'{int(confidence*100)}%'
            cv2.putText(frame, text, (x1,y1-5), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.5, (0,0,255),2)   
    
    
    cv2.imshow('test', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()